Let's create test data: a batch of three 8-channel signals of different length.

In [1]:
import torch

n_channel = 8

x1 = torch.rand((1000, n_channel))  # Length: 1000
x2 = torch.rand((2000, n_channel))  # Length: 2000
x3 = torch.rand((3000, n_channel))  # Length: 3000

Then gather the signals, pad them to the batch max length, store the original signal lengths, create a mask to recover later which elements are padded.

In [2]:
from dataloading import collate_fn

x_in, lengths, mask = collate_fn([(x1, 1000), (x2, 2000), (x3, 3000)])
print('Input tensor size:', x_in.size())

Input tensor size: torch.Size([3, 3000, 8])


Define the autoencoder model.

In [3]:
from modeling import SignalAutoencoder

n_conv_channel_1 = 64
n_conv_channel_2 = 128
n_conv_channel_3 = 256
lstm_hidden_size = 512
n_lstm_layer = 2

autoenc = SignalAutoencoder(n_channel, n_conv_channel_1,
                            n_conv_channel_2, n_conv_channel_3,
                            lstm_hidden_size, n_lstm_layer)

In [4]:
x_out = autoenc(x_in)

print('Output tensor size:', x_out.size())

Output tensor size: torch.Size([3, 3000, 8])


Compute the L1 distance between the original input and the reconstructed output. Earlier variables `lengths` and `mask` are used here.

In [5]:
from modeling import sequence_l1

err = sequence_l1(x_in, x_out, lengths, mask)

print('L1 distance:', err.item())

L1 distance: 3.994481325149536


This distance can be used as a loss for training the autoencoder.

Next, get the codes (compressed representations) corresponding to the batch input.

In [6]:
code = autoenc.encode(x_in)
print('Code size:', code.size())

Code size: torch.Size([3, 1024])


The latent space dimension, i.e. the code size, is `lstm_hidden_size * n_lstm_layer`, here `512 * 2 = 1024`.